# Word Embeddings e Sentiment Analysis (PyTorch puro)

Este notebook demonstra como representar palavras por vetores densos (*word embeddings*) e usá-los em uma tarefa de classificação de sentimentos usando apenas PyTorch.

Etapas:
1. Preparar um pequeno corpus rotulado (positivo/negativo)
2. Tokenizar e mapear palavras para índices
3. Criar embeddings aprendíveis
4. Treinar um classificador simples
5. Avaliar e testar previsões

In [1]:
# pip install torch torchvision torchaudio

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/ebezerra/miniconda3/envs/gcc1626/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/ebezerra/miniconda3/envs/gcc1626/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/ebezerra/miniconda3/envs/gcc1626/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in

## 1. Conjunto de dados simples

In [3]:
corpus = [
    ("o filme foi ótimo e emocionante", 1),
    ("adorei o enredo e os personagens", 1),
    ("a atuação foi excelente", 1),
    ("o filme é terrível e cansativo", 0),
    ("péssimo roteiro e trilha sonora ruim", 0),
    ("não gostei do final", 0),
]

## 2. Tokenização e vocabulário

In [4]:
def tokenize(text):
    return text.lower().split()

vocab = {"<pad>": 0, "<unk>": 1}
for sentence, _ in corpus:
    for tok in tokenize(sentence):
        if tok not in vocab:
            vocab[tok] = len(vocab)

vocab_size = len(vocab)
print("Vocabulário:", vocab)

def encode(sentence, max_len=8):
    tokens = tokenize(sentence)
    ids = [vocab.get(t, 1) for t in tokens]
    if len(ids) < max_len:
        ids += [0] * (max_len - len(ids))
    else:
        ids = ids[:max_len]
    return ids

max_len = 8
encoded = [(encode(s, max_len), y) for s, y in corpus]
print(encoded[:2])

Vocabulário: {'<pad>': 0, '<unk>': 1, 'o': 2, 'filme': 3, 'foi': 4, 'ótimo': 5, 'e': 6, 'emocionante': 7, 'adorei': 8, 'enredo': 9, 'os': 10, 'personagens': 11, 'a': 12, 'atuação': 13, 'excelente': 14, 'é': 15, 'terrível': 16, 'cansativo': 17, 'péssimo': 18, 'roteiro': 19, 'trilha': 20, 'sonora': 21, 'ruim': 22, 'não': 23, 'gostei': 24, 'do': 25, 'final': 26}
[([2, 3, 4, 5, 6, 7, 0, 0], 1), ([8, 2, 9, 6, 10, 11, 0, 0], 1)]


## 3. Dataset e DataLoader

In [5]:
class SentimentDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x), torch.tensor(y, dtype=torch.float32)

dataset = SentimentDataset(encoded)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

## 4. Modelo com embeddings aprendíveis

In [6]:
class EmbeddingClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.fc1 = nn.Linear(emb_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.act = nn.ReLU()
        self.out = nn.Sigmoid()

    def forward(self, x):
        emb = self.emb(x)
        mean_emb = emb.mean(dim=1)
        h = self.act(self.fc1(mean_emb))
        y = self.out(self.fc2(h))
        return y.squeeze()

model = EmbeddingClassifier(vocab_size, emb_dim=16, hidden_dim=8)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

## 5. Treinamento

In [7]:
for epoch in range(25):
    total_loss = 0
    for xb, yb in loader:
        optimizer.zero_grad()
        preds = model(xb) # forward pass
        loss = criterion(preds, yb) # loss
        loss.backward() # backpropagation
        optimizer.step() # parameter update
        total_loss += loss.item()
    if (epoch+1) % 5 == 0:
        print(f"Época {epoch+1:02d} | Loss: {total_loss/len(loader):.4f}")

Época 05 | Loss: 0.6770
Época 10 | Loss: 0.6087
Época 15 | Loss: 0.4520
Época 20 | Loss: 0.2313
Época 25 | Loss: 0.0836


## 6. Teste com novas frases

In [8]:
def predict(sentence):
    x = torch.tensor(encode(sentence, max_len)).unsqueeze(0)
    with torch.no_grad():
        p = model(x).item()
    label = "positivo" if p >= 0.5 else "negativo"
    return f"{sentence} → {label} ({p:.2f})"

for s in [
    "amei o filme e a trilha sonora",
    "o roteiro é fraco e entediante",
    "personagens excelentes e cativantes",
    "não recomendo o filme",
]:
    print(predict(s))

amei o filme e a trilha sonora → negativo (0.11)
o roteiro é fraco e entediante → negativo (0.14)
personagens excelentes e cativantes → negativo (0.43)
não recomendo o filme → negativo (0.34)


## 7. Visualizando embeddings

In [10]:
emb_weights = model.emb.weight.detach()
print("Dimensão:", emb_weights.shape)

def closest(word, topn=5):
    if word not in vocab: 
        return []
    idx = vocab[word]
    wv = emb_weights[idx]
    # similaridade de cosseno em PyTorch
    sims = torch.nn.functional.cosine_similarity(emb_weights, wv.unsqueeze(0))
    best = torch.argsort(sims, descending=True)[1:topn+1]
    inv_vocab = {i: w for w, i in vocab.items()}
    return [(inv_vocab[int(i)], float(sims[i])) for i in best]

print("Mais próximos de 'filme':", closest("filme"))


Dimensão: torch.Size([27, 16])
Mais próximos de 'filme': [('é', 0.4726399779319763), ('sonora', 0.38141706585884094), ('e', 0.2239847481250763), ('roteiro', 0.12832169234752655), ('trilha', 0.11593659222126007)]
